<a href="https://colab.research.google.com/github/helloworld53/projects/blob/main/Named_Entity_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install the transformers library
!pip install transformers

In [ ]:
!pip install --upgrade accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 3.8 MB/s eta 0:00:00


In [ ]:
!pip install datasets transformers huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.3 MB/s eta 0:00:00


In [ ]:
# Import required packages
import torch
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer

# Create class for data preparation
class SimpleDataset:
    def __init__(self, tokenized_texts):
        self.tokenized_texts = tokenized_texts

    def __len__(self):
        return len(self.tokenized_texts["input_ids"])

    def __getitem__(self, idx):
        return {k: v[idx] for k, v in self.tokenized_texts.items()}

In [ ]:
df = pd.read_csv("train.csv")
df_val = pd.read_csv("val.csv")
df_test = pd.read_csv("test.csv")


In [ ]:
separator_token = "[SEP]"
df['text'] = df.iloc[:, :-1].apply(lambda x: separator_token.join(x.dropna().astype(str)), axis=1)
df_val['text'] = df_val.iloc[:, :-1].apply(lambda x: separator_token.join(x.dropna().astype(str)), axis=1)
df_test['text'] = df_test.iloc[:, :-1].apply(lambda x: separator_token.join(x.dropna().astype(str)), axis=1)

In [ ]:
df = pd.concat([df['text'] , df.label], axis =1 )
df_val = pd.concat([df_val['text'] , df_val.label], axis =1 )
df_val

,text,label
0,"387733352[SEP][""Skin Care"",""Face"",""Makeup Remo...",False
1,"3382959[SEP][""Makeup"",""Face""][SEP]["" Makeup"",""...",False
2,"95788062[SEP][""超級市場"",""零食 甜品"",""韓國零食""][SEP][""sup...",True
3,"81595972[SEP][""Lebensmittel & Getränke"",""Eier ...",False
4,"92325406[SEP][""食品饮料"",""饮料冲调"",""牛奶乳品""][SEP][""food...",True
...,...,...
595,"85988180[SEP][""Health & Household"",""Vitamins &...",True
596,"47278044[SEP][""Nabiał i jaja"",""Masło i margary...",False
597,"481588437[SEP][""Categories"",""Supplements"",""Bra...",True
598,"324468872[SEP][""Arzneimittel &amp; weitere Pro...",True


In [ ]:
df.label = [0 if x == False else 1 for x in df.label]
df_val.label = [0 if x == False else 1 for x in df_val.label]

In [ ]:
#Preprocessing Data for Eval
# X_train,X_test, y_train , y_test = train_test_split(
# ...     df.title, df.sentiment, test_size=0.20, random_state=42)

# df_train = pd.concat([X_train, y_train], axis=1)
# df_val = pd.concat([X_test, y_test], axis=1)
# df_train.reset_index(drop = True,inplace = True)
# df_val.reset_index(drop = True,inplace = True)
# df_train = df_train.rename(columns={"title": "text", "sentiment" : "label"})
# df_val = df_val.rename(columns={"title": "text", "sentiment" : "label"})

In [ ]:
# Load tokenizer and model, create trainer , train , evaluate and then predict.
model_name = "Davlan/distilbert-base-multilingual-cased-ner-hrl"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at Davlan/distilbert-base-multilingual-cased-ner-hrl and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from datasets import Dataset
# # with eval
# dataset_train = Dataset.from_pandas(df_train)
# dataset_test  = Dataset.from_pandas(df_val)
# #without eval
dataset = Dataset.from_pandas(df)
dataset_v = Dataset.from_pandas(df_val)

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)
# #with eval
# tokenized_train = dataset_train.map(preprocess_function, batched=True)
# tokenized_test = dataset_test.map(preprocess_function, batched=True)
# #without eval
# tokenized_data = dataset.map(preprocess_function, batched=True)
tokenized_data = dataset.map(preprocess_function, batched=True)
tokenized_val = dataset_v.map(preprocess_function, batched=True)

Map:   0%|          | 0/5831 [00:00<?, ? examples/s]

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
# from datasets import load_metric

# def compute_metrics(eval_pred):
#     load_accuracy = load_metric("accuracy")
#     load_f1 = load_metric("f1")

#     logits, labels = eval_pred
#     predictions = np.argmax(logits, axis=-1)
#     accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
#     f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
#     return {"accuracy": accuracy, "f1": f1}

In [ ]:
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(
    output_dir="/content/model_checkpoints",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    warmup_steps = 500,

)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data,
    eval_dataset = df_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,0.669200
1000,0.617300
1500,0.571200
2000,0.486200


TrainOutput(global_step=2187, training_loss=0.5762716606429684, metrics={'train_runtime': 1049.5635, 'train_samples_per_second': 16.667, 'train_steps_per_second': 2.084, 'total_flos': 2317518708337008.0, 'train_loss': 0.5762716606429684, 'epoch': 3.0})

In [ ]:
trainer.evaluate()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /usr/local/lib/python3.10/dist-packages/pandas/core/indexes/base.py:3802 in get_loc              │
│                                                                                                  │
│   3799 │   │   │   │   )                                                                         │
│   3800 │   │   │   casted_key = self._maybe_cast_indexer(key)                                    │
│   3801 │   │   │   try:                                                                          │
│ ❱ 3802 │   │   │   │   return self._engine.get_loc(casted_key)                                   │
│   3803 │   │   │   except KeyError as err:                                                       │
│   3804 │   │   │   │   raise KeyError(key) from err                                              │
│   3805 │   │   │   except TypeError:                                                             │
│                                                                                                  │
│ in pandas._libs.index.IndexEngine.get_loc:138                                                    │
│                                                                                                  │
│ in pandas._libs.index.IndexEngine.get_loc:165                                                    │
│                                                                                                  │
│ in pandas._libs.hashtable.PyObjectHashTable.get_item:5745                                        │
│                                                                                                  │
│ in pandas._libs.hashtable.PyObjectHashTable.get_item:5753                                        │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyError: 0

The above exception was the direct cause of the following exception:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:3029 in evaluate                 │
│                                                                                                  │
│   3026 │   │   start_time = time.time()                                                          │
│   3027 │   │                                                                                     │
│   3028 │   │   eval_loop = self.prediction_loop if self.args.use_legacy_prediction_loop else se  │
│ ❱ 3029 │   │   output = eval_loop(                                                               │
│   3030 │   │   │   eval_dataloader,                                                              │
│   3031 │   │   │   description="Evaluation",                                                     │
│   3032 │   │   │   # No point gathering the predictions if there are no metrics, otherwise we d  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:3200 in evaluation_loop          │
│                                                                                                  │
│   3197 │   │                                                                                     │
│   3198 │   │   observed_num_examples = 0                                                         │
│   3199 │   │   # Main evaluation loop                                                            │
│ ❱ 3200 │   │   for step, inputs in enumerate(dataloader):                                        │
│   3201 │   │   │   # Update the observed num examples                        

##Prediction

In [ ]:
# Load tokenizer and model, create trainer (without training  / direct prediction COMMENTED)
# model_name = "siebert/sentiment-roberta-large-english"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForSequenceClassification.from_pretrained(model_name)
# trainer = Trainer(model = model)

In [ ]:
# df_test = pd.read_csv("test (1).csv")
pred_texts = df_test['text'].dropna().astype('str').tolist()

In [ ]:
# Tokenize texts and create prediction data set
tokenized_texts = tokenizer(pred_texts,truncation=True,padding=True)
pred_dataset = SimpleDataset(tokenized_texts)

In [ ]:
# Run predictions
predictions = trainer.predict(pred_dataset)

In [ ]:
# Transform predictions to labels
preds = predictions.predictions.argmax(-1)
labels = pd.Series(preds).map(model.config.id2label)
scores = (np.exp(predictions[0])/np.exp(predictions[0]).sum(-1,keepdims=True)).max(1)

In [ ]:
# Create DataFrame with texts, predictions, labels, and scores
dfs = pd.DataFrame(list(zip(pred_texts,preds,labels,scores)), columns=['text','pred','label','score'])
# df_submit = pd.DataFrame(dfs.pred)
# df_submit.index.names = ['sku_id']
# df_submit = df_submit.rename(columns={"pred": "label"})
# df_submit.head()
dfs

,text,pred,label,score
0,"77814155[SEP][""Grocery"",""Food Cupboard"",""Cerea...",0,O,0.888381
1,"94821179[SEP][""Bellezza"",""Cura della pelle"",""C...",0,O,0.651114
2,"162080129[SEP][""RedMart"",""Groceries"",""Dairy & ...",1,B-DATE,0.885624
3,"67013430[SEP][""Inicio"",""Maquillaje"",""Rostro"",""...",0,O,0.570940
4,"24022928[SEP][""Início"",""Óleos & Temperos"",""Tem...",1,B-DATE,0.644001
...,...,...,...,...
1113,"197055456[SEP][""Menu 1 element (kopia)"",""Karmy...",1,B-DATE,0.734857
1114,"341727825[SEP][""홈"",""건강"",""건강식품"",""영양제"",""SAM-E메티오...",1,B-DATE,0.912290
1115,"102847743[SEP][""홈"",""건강식품"",""성분별 건강식품""][SEP][""ho...",1,B-DATE,0.907658
1116,"364281952[SEP][""Baby Products"",""Feeding"",""Baby...",1,B-DATE,0.844539


In [ ]:
import re
dfs['sku_id'] = dfs['text'].apply(lambda x: re.findall(r'\d+', x)[0] if re.findall(r'\d+', x) else None)
dfrr = pd.DataFrame([dfs.sku_id, dfs.pred])
dfrr = dfrr.T
dfrr.set_index('sku_id', inplace= True)
dfrr = dfrr.rename(columns={"pred": "label"})
dfrr.head()

,label
sku_id,
77814155,0
94821179,0
162080129,1
67013430,0
24022928,1


In [ ]:
# dfrr.pred = dfrr.pred.astype(bool)

In [ ]:
dfrr.to_csv('df_submit_neww_.csv', index = True)

In [ ]:
#df_pred = pd.read_csv(file_name)
#pred_texts = df_pred[text_column].dropna().astype('str').tolist()